In [1]:
import csv
import pandas
import numpy
import random
from CWHashingFamily import CWHashingFamily

SHINGLES_LEN = 4
PRIME = 997
BINS = 900

def shingles(text, shingleSize):
    return [text[i:i+shingleSize] for i in xrange(0,len(text)-(shingleSize+1))]


def minhash(shinglesList, seed):
    random.seed(seed)
    minhash = -1
    for shingle in shinglesList:
        cwFactory = CWHashingFamily(BINS, PRIME)
        numericHash = cwFactory.numericFunction(random.randrange(1,PRIME), random.randrange(0,PRIME))
        stringHash = cwFactory.stringFunction(random.randrange(0,PRIME),numericHash)
        shHash = stringHash.hash(shingle)
        if ((minhash == -1) or (shHash < minhash)):
            minhash = shHash
    return minhash

In [11]:
ID = 0
TEXT = 9
R = 4
B = 8

trainRDD = sc.textFile("train_sin_repeticiones.csv").map(lambda line: line.split(","))

trainRDD = trainRDD.filter(lambda line: line[ID] != "Id")

trainRDD = trainRDD.map(lambda line: (line[ID], shingles(str(line[TEXT]),SHINGLES_LEN)))

trainRDD = trainRDD.map(lambda line: (line[0], [ minhash(line[1], i) for i in xrange(0, R*B) ] )).take(5)

print trainRDD


[(u'492346', [3, 2, 0, 9, 1, 0, 1, 3, 6, 0, 0, 5, 14, 9, 0, 0, 0, 0, 7, 1, 3, 0, 1, 0, 2, 0, 0, 0, 3, 2, 0, 3]), (u'343811', [15, 5, 6, 3, 1, 3, 3, 0, 4, 0, 1, 2, 1, 0, 4, 6, 0, 4, 4, 1, 0, 5, 1, 1, 5, 5, 0, 3, 0, 5, 1, 2]), (u'347257', [20, 1, 3, 19, 24, 11, 8, 0, 33, 10, 2, 3, 21, 19, 10, 8, 2, 10, 12, 24, 0, 5, 24, 25, 1, 5, 6, 28, 0, 1, 3, 9]), (u'225529', [25, 29, 67, 38, 64, 6, 0, 42, 85, 39, 37, 2, 14, 18, 2, 73, 82, 2, 0, 125, 240, 21, 39, 3, 23, 2, 17, 68, 32, 29, 45, 20]), (u'485488', [1, 1, 12, 0, 0, 7, 3, 0, 5, 0, 1, 4, 0, 0, 5, 0, 0, 5, 5, 0, 1, 3, 0, 2, 1, 3, 5, 5, 0, 1, 3, 3])]


In [44]:
from LSH import LSH
from Record import Record

lsh = LSH(R,B)

for i in xrange(len(trainRDD)):
    record = Record((trainRDD[i][0], trainRDD[i][1]))
    lsh.add(record)


Adding record:492346 to LSH at POS:57
table 0 :[u'492346']
Adding record:492346 to LSH at POS:68
table 1 :[u'492346']
Adding record:492346 to LSH at POS:7
table 2 :[u'492346']
Adding record:492346 to LSH at POS:87
table 3 :[u'492346']
Adding record:492346 to LSH at POS:42
table 4 :[u'492346']
Adding record:492346 to LSH at POS:38
table 5 :[u'492346']
Adding record:492346 to LSH at POS:31
table 6 :[u'492346']
Adding record:492346 to LSH at POS:53
table 7 :[u'492346']
Adding record:343811 to LSH at POS:46
table 0 :[u'343811']
Adding record:343811 to LSH at POS:40
table 1 :[u'343811']
Adding record:343811 to LSH at POS:50
table 2 :[u'343811']
Adding record:343811 to LSH at POS:51
table 3 :[u'343811']
Adding record:343811 to LSH at POS:93
table 4 :[u'343811']
Adding record:343811 to LSH at POS:89
table 5 :[u'343811']
Adding record:343811 to LSH at POS:21
table 6 :[u'343811']
Adding record:343811 to LSH at POS:56
table 7 :[u'343811']
Adding record:347257 to LSH at POS:44
table 0 :[u'347257'

In [50]:
shinglesText = ['"i g', 'i go', ' got', 'got ', 'ot t', 't th', ' the', 'them', 'hem ', 'em i', 'm in', ' in ', 'in a', 'n a ', ' a v', 'a ve', ' ver', 'very', 'ery ', 'ry t', 'y ti', ' tim', 'time', 'imel', 'mely', 'ely ', 'ly m', 'y ma', ' man', 'mann', 'anne', 'nner', 'ner ', 'er a', 'r an', ' and', 'and ', 'nd t', 'd th', ' the', 'they', "hey'", "ey'r", "y're", "'re ", 're a', 'e al', ' all', 'all ', 'll v', 'l ve', ' ver', 'very', 'ery ', 'ry l', 'y la', ' lar', 'larg', 'arge', 'rge ', 'ge a', 'e an', ' and', 'and ', 'nd g', 'd gr', ' gre', 'gree', 'reen', 'een.', 'en. ', 'n. t', '. th', ' the', 'ther', 'here', 'ere ', 're s', 'e se', ' see', 'seem', 'eems', 'ems ', 'ms t', 's to', ' to ', 'to b', 'o be', ' be ', 'be a', 'e an', ' an ', 'an i', 'n in', ' ino', 'inor', 'nord', 'ordi', 'rdin', 'dina', 'inat', 'nate', 'ate ', 'te a', 'e am', ' amo', 'amou', 'moun', 'ount', 'unt ', 'nt o', 't of', ' of ', 'of t', 'f th', ' the', 'the ', 'he s', 'e se', ' see', 'seed', 'eeds', 'eds ', 'ds e', 's e ', ' e o', 'e ou', ' out', 'out ', 'ut a', 't a ', ' a w', 'a we', ' wei', 'weir', 'eird', 'ird ', 'rd b', 'd be', ' bei', 'beig', 'eige', 'ige ', 'ge c', 'e co', ' col']
test = (492346,shinglesText)
print test
test = (test[0], [ minhash(test[1], i) for i in xrange(0, R*B) ])
testRecord = Record(test)
print test



(492346, ['"i g', 'i go', ' got', 'got ', 'ot t', 't th', ' the', 'them', 'hem ', 'em i', 'm in', ' in ', 'in a', 'n a ', ' a v', 'a ve', ' ver', 'very', 'ery ', 'ry t', 'y ti', ' tim', 'time', 'imel', 'mely', 'ely ', 'ly m', 'y ma', ' man', 'mann', 'anne', 'nner', 'ner ', 'er a', 'r an', ' and', 'and ', 'nd t', 'd th', ' the', 'they', "hey'", "ey'r", "y're", "'re ", 're a', 'e al', ' all', 'all ', 'll v', 'l ve', ' ver', 'very', 'ery ', 'ry l', 'y la', ' lar', 'larg', 'arge', 'rge ', 'ge a', 'e an', ' and', 'and ', 'nd g', 'd gr', ' gre', 'gree', 'reen', 'een.', 'en. ', 'n. t', '. th', ' the', 'ther', 'here', 'ere ', 're s', 'e se', ' see', 'seem', 'eems', 'ems ', 'ms t', 's to', ' to ', 'to b', 'o be', ' be ', 'be a', 'e an', ' an ', 'an i', 'n in', ' ino', 'inor', 'nord', 'ordi', 'rdin', 'dina', 'inat', 'nate', 'ate ', 'te a', 'e am', ' amo', 'amou', 'moun', 'ount', 'unt ', 'nt o', 't of', ' of ', 'of t', 'f th', ' the', 'the ', 'he s', 'e se', ' see', 'seed', 'eeds', 'eds ', 'ds e'

In [51]:
print lsh.getAllSimilarRecords(testRecord)

obtained pos:57 for record (492346,[3, 2, 0, 9, 1, 0, 1, 3, 6, 0, 0, 5, 14, 9, 0, 0, 0, 0, 7, 1, 3, 0, 1, 0, 2, 0, 0, 0, 3, 2, 0, 3])at table 0
Getting records from POS:57 from: [u'492346']
obtained pos:68 for record (492346,[3, 2, 0, 9, 1, 0, 1, 3, 6, 0, 0, 5, 14, 9, 0, 0, 0, 0, 7, 1, 3, 0, 1, 0, 2, 0, 0, 0, 3, 2, 0, 3])at table 1
Getting records from POS:68 from: [u'492346']
obtained pos:7 for record (492346,[3, 2, 0, 9, 1, 0, 1, 3, 6, 0, 0, 5, 14, 9, 0, 0, 0, 0, 7, 1, 3, 0, 1, 0, 2, 0, 0, 0, 3, 2, 0, 3])at table 2
Getting records from POS:7 from: [u'492346']
obtained pos:87 for record (492346,[3, 2, 0, 9, 1, 0, 1, 3, 6, 0, 0, 5, 14, 9, 0, 0, 0, 0, 7, 1, 3, 0, 1, 0, 2, 0, 0, 0, 3, 2, 0, 3])at table 3
Getting records from POS:87 from: [u'492346']
obtained pos:42 for record (492346,[3, 2, 0, 9, 1, 0, 1, 3, 6, 0, 0, 5, 14, 9, 0, 0, 0, 0, 7, 1, 3, 0, 1, 0, 2, 0, 0, 0, 3, 2, 0, 3])at table 4
Getting records from POS:42 from: [u'492346']
obtained pos:38 for record (492346,[3, 2, 0, 9, 1, 0

4
3
4
3
1
10
10
1
